# Анализ вакансии для аналитиков данных

- Проект: Анализ вакансии для аналитиков данных
- Цель: Визуализировать информацию о рынке вакансий для аналитиков в Европе

## План действий

1. Распарсить предоставленный csv файл с помощью BS 4, создав следующие признаки:
- наименование вакансии
- город
- страна
- тип занятости (online, hybride, on-site)
- компания
- размер компании (количество работников)
- сфера деятельности компании
- требуемые хард скилы
- дата публикации вакансии
- количество кандидатов на вакансию
2. Подготовка данных
  - Определить требуемые хард скилы и тип занятости (online, hybride, on-site) для каждой вакансии.
3. Подготовка данных к визуализации
  - Проверить данные на дубликаты и не релевантные заданию вакансии.
  - Удалить ненужные атрибуты (признаки).
4. Визуализация данных
  - Построить интерактивный дашборд в любой из BI систем.

## Подготовка данных

In [ ]:
# Импортируем библиотеки:
import pandas as pd
from bs4 import BeautifulSoup
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
from scipy import stats as st
import re
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Закрепим формат float:
pd.set_option('display.float_format', '{:,.2f}'.format)

# Уберем предупреждения:
import warnings
warnings.simplefilter('ignore')

In [ ]:
# Загрузим датасет из облака
!gdown 1Q5RyXLmjhJ8L55ya9F2CP95Whq7UZ6q3

Downloading...
From: https://drive.google.com/uc?id=1Q5RyXLmjhJ8L55ya9F2CP95Whq7UZ6q3
To: /content/masterskaya_parsing_LinkedIn_2023_05_23.csv
100% 28.5M/28.5M [00:00<00:00, 220MB/s]


In [ ]:
# Сохраним данные из датасетов:
df = pd.read_csv('masterskaya_parsing_LinkedIn_2023_05_23.csv', index_col=0)

In [ ]:
# Выведем первую вакансию
display(HTML(df['html'][3]))

In [ ]:
# Потестируем парсинг данных на первой вакансии, потом спарсим все через фукнцию
html = df['html'][0]

In [ ]:
# Применим BeautifulSoup
soup = BeautifulSoup(html)

In [ ]:
# Получим наименование вакансии
title = soup.find('h2', class_ = 't-24 t-bold jobs-unified-top-card__job-title').text.strip()
title

'Data Analyst'

In [ ]:
# Получим город
city = soup.find('span', class_ = 'jobs-unified-top-card__bullet').text.strip().split()[0].rstrip(',')
city

'Basel'

In [ ]:
# Получим страну
country = soup.find('span', class_ = 'jobs-unified-top-card__bullet').text.strip().split(',')[-1]
country

' Switzerland'

In [ ]:
# тип занятости (online, hybride, on-site)
employment_type = soup.find('span', class_ = 'jobs-unified-top-card__workplace-type').text.strip()
employment_type

'On-site'

In [ ]:
# Получим компанию
company_name = soup.find('a', class_ = 'ember-view t-black t-normal').text.strip()
company_name

'PharmiWeb.Jobs: Global Life Science Jobs'

In [ ]:
# размер компании (количество работников)
employee_qty = soup.find('span', class_ = 'jobs-company__inline-information').text.strip()
employee_qty

'11-50 employees'

In [ ]:
# сфера деятельности компании
company_field = soup.find('div', class_ = 't-14 mt5').text.strip().split('\n')[0]
company_field

'Staffing & Recruiting'

In [ ]:
# Список скилов
skills = [
    'a/b testing', 'ab testing', 'actian', 'adobe analytics', 'adobe audience manager',
    'adobe experience platform', 'adobe launch', 'adobe target', 'ai', 'airflow',
    'alooma', 'alteryx', 'amazon machine learning', 'amazon web services', 'aml',
    'amplitude', 'ansible', 'apache camel', 'apache nifi', 'apache spark',
    'api', 'asana', 'auth0', 'aws', 'aws glue', 'azure', 'azure data factory',
    'basecamp', 'bash', 'beats', 'big query', 'bigquery', 'birst', 'bitbucket',
    'blendo', 'bootstrap', 'business objects bi', 'c#', 'c++', 'caffe', 'cassandra',
    'cdata sync', 'chronograf', 'ci/cd', 'cicd', 'clickhouse', 'cloudera', 'cluvio',
    'cntk', 'cognos', 'composer', 'computer vision', 'conda', 'confluence',
    'couchbase', 'css', 'd3.js', 'dash', 'dashboard', 'data factory', 'data fusion',
    'data mining', 'data studio', 'data warehouse', 'databricks', 'dataddo',
    'dataflow', 'datahub', 'dataiku', 'datastage', 'dbconvert', 'dbeaver', 'dbt',
    'deep learning', 'dl/ml', 'docker', 'domo', 'dune', 'dv360', 'dynamodb',
    'elasticsearch', 'elt', 'erwin', 'etl', 'etleap', 'excel', 'facebook business manager',
    'fivetran', 'fuzzy', 'ga360', 'gcp', 'gensim', 'ggplot', 'git', 'github', 'gitlab',
    'google ads', 'google analytics', 'google cloud platform', 'google data flow',
    'google optimize', 'google sheets', 'google tag manager', 'google workspace',
    'grafana', 'hadoop', 'hana', 'hanagrafana', 'hbase', 'hdfs', 'hevo data', 'hightouch',
    'hive', 'hivedatabricks', 'html', 'hubspot', 'ibm coremetrics', 'inetsoft',
    'influxdb', 'informatica', 'integrate.io', 'iri voracity', 'izenda', 'java',
    'java script', 'javascript', 'jenkins', 'jira', 'jmp', 'julia', 'jupyter',
    'k2view', 'kafka', 'kantar', 'kapacitor', 'keras', 'kibana', 'kubernetes',
    'lambda', 'linux', 'logstash', 'looker', 'lstm', 'luidgi', 'matillion', 'matlab',
    'matplotlib', 'mendix', 'metabase', 'microsoft sql', 'microsoft sql server',
    'microstrategy', 'miro', 'mixpanel', 'ml', 'ml flow', 'mlflow', 'mongodb', 'mxnet',
    'mysql', 'natural nanguage processing', 'neo4j', 'nlp', 'nltk', 'nosql', 'numpy',
    'oauth', 'octave', 'omniture', 'omnituregitlab', 'openshift', 'openstack',
    'optimizely', 'oracle', 'oracle business intelligence', 'oracle data integrator',
    'pandas', 'panorama', 'pentaho', 'plotly', 'postgre', 'postgresql', 'posthog',
    'power amc', 'power bi', 'power point', 'powerbi', 'powerpivot', 'powerpoint',
    'powerquery', 'pyspark', 'python', 'pytorch', 'pytorchhevo data', 'qlik',
    'qlik sense', 'qlikview', 'querysurge', 'r', 'raphtory', 'rapidminer', 'redash',
    'redis', 'redshift', 'retool', 'rivery', 'rust', 's3', 'sa360', 'salesforce', 'sap',
    'sap business objects', 'sas', 'sas visual analytics', 'scala', 'scikit-learn',
    'scipy', 'seaborn', 'segment', 'selenium', 'sem rush', 'semrush', 'shell', 'shiny',
    'singer', 'sisense', 'skyvia', 'snowflake', 'spacy', 'spark', 'sparkml', 'splunk',
    'spotfire', 'spreadsheet', 'spss', 'sql', 'ssis', 'sssr', 'stambia', 'statistics',
    'statsbot', 'stitch', 'streamlit', 'streamsets', 'svn', 't-sql', 'tableau', 'talend',
    'targit', 'tealium', 'telegraf', 'tensorflow', 'terraapi', 'terraform', 'theano',
    'thoughtspot', 'timeseries', 'trello', 'unix', 'vba', 'vtom', 'webfocus', 'wfh',
    'xplenty', 'xtract.io', 'yellowfin'
]

In [ ]:
# Функция для получения скиллов
def skills_finder(cell, skill_list_2=skills):
    matched_skills_list=[]
    for i in skill_list_2:
        if i == 'c++':
          if re.search('\Wc\+\+\W', cell.lower()):
            matched_skills_list.append(i)
        # word_border + rewritten "i" in special symbols + word_border
        else:
          pattern = (
            r'(\b|\W)'
            + re.escape(i)
            + r'(\b|\W)'
            +'|'
            + r'(\b|\W)'
            +re.escape(i.replace(' ', ''))
            + r'(\b|\W)'
        )
          if re.search(pattern, cell.lower()):
              matched_skills_list.append(i)
    return matched_skills_list

In [ ]:
# Описание вакансии
job_description = soup.find('div', class_ = 'jobs-description__content jobs-description-content').text.strip()

# Применим функцию к 'job_description'
skills = skills_finder(job_description)

# Выведем их
print(skills)

['data mining', 'excel', 'sap', 'sas', 'spss', 'sql', 'statistics']


In [ ]:
# дата публикации вакансии
date = soup.find('span', class_ = 'jobs-unified-top-card__posted-date').text.strip()
date

'1 week ago'

In [ ]:
# количество кандидатов на вакансию
applicants = soup.find('span', class_ = 'jobs-unified-top-card__applicant-count').text.strip().split()[0]
applicants

'47'

In [ ]:
# напишем функцию для парсинга всех данных
def extract_data(df):
    # Создадим пустой список
    data = []

    # Пройдем по всем данным
    for html in df['html']:
        # Создадим объект BeautifulSoup object
        soup = BeautifulSoup(html, 'html.parser')

        # Получим данные
        try:
            title = soup.find('h2', class_ = 't-24 t-bold jobs-unified-top-card__job-title').text.strip()
        except AttributeError:
            title = np.nan
        try:
            city = soup.find('span', class_ = 'jobs-unified-top-card__bullet').text.strip().split(',')[0]
        except AttributeError:
            city = np.nan
        try:
            country = soup.find('span', class_ = 'jobs-unified-top-card__bullet').text.strip().split(',')[-1]
        except AttributeError:
            country = np.nan
        try:
            employment_type = soup.find('span', class_ = 'jobs-unified-top-card__workplace-type').text.strip()
        except AttributeError:
            employment_type = np.nan
        try:
            company_name = soup.find('a', class_ = 'ember-view t-black t-normal').text.strip()
        except AttributeError:
            company_name = np.nan
        try:
            employee_qty = soup.find('span', class_ = 'jobs-company__inline-information').text.strip()
        except AttributeError:
            employee_qty = np.nan
        try:
            company_field = soup.find('div', class_ = 't-14 mt5').text.strip().split('\n')[0]
        except AttributeError:
            company_field = np.nan
        try:
            skills = skills_finder(soup.find('div', class_ = 'jobs-description__content jobs-description-content').text.strip())
        except AttributeError:
            skills = np.nan
        try:
            date = soup.find('span', class_ = 'jobs-unified-top-card__posted-date').text.strip()
        except AttributeError:
            date = np.nan
        try:
            applicants = soup.find('span', class_ = 'jobs-unified-top-card__applicant-count').text.strip().split()[0]
        except AttributeError:
            applicants = np.nan

        # Добавим данные в список
        data.append([title, city, country, employment_type, company_name, employee_qty, company_field, skills, date, applicants])

    # Конвертируем список в dataframe
    result_df = pd.DataFrame(data, columns=['title', 'city', 'country', 'employment_type', 'company_name', 'employee_qty', 'company_field', 'skills', 'date', 'applicants'])

    return result_df


In [ ]:
# применим функцию на данных
result_df = extract_data(df)
result_df.head()

,title,city,country,employment_type,company_name,employee_qty,company_field,skills,date,applicants
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting,"[data mining, excel, sap, sas, spss, sql, stat...",1 week ago,47
1,Data Analyst - Logistics,Coventry,United Kingdom,On-site,NaN,NaN,NaN,[],1 week ago,NaN
2,Data Analyst - Logistics,Coventry,United Kingdom,On-site,NaN,NaN,NaN,[wfh],1 week ago,NaN
3,Data Analyst (Space & Planning),South Molton,United Kingdom,On-site,NaN,NaN,NaN,[excel],1 week ago,NaN
4,Data Analyst,Lugano,Switzerland,On-site,NaN,NaN,NaN,"[aws, data warehouse, etl, gcp, oracle, oracle...",2 weeks ago,NaN


In [ ]:
# Выведем тип данных
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            998 non-null    object
 1   city             998 non-null    object
 2   country          998 non-null    object
 3   employment_type  930 non-null    object
 4   company_name     969 non-null    object
 5   employee_qty     964 non-null    object
 6   company_field    964 non-null    object
 7   skills           998 non-null    object
 8   date             998 non-null    object
 9   applicants       838 non-null    object
dtypes: object(10)
memory usage: 78.1+ KB


In [ ]:
# Поменяем applicants на int64, чтобы учесть пропуски
result_df['applicants'] = result_df['applicants'].astype('Int64')

In [ ]:
# Посмотрим на даты
result_df['date'].unique()

array(['1 week ago', '2 weeks ago', '6 days ago', '3 weeks ago',
       '2 days ago', '1 day ago', '4 days ago', '4 weeks ago',
       '3 days ago', '5 days ago', '12 minutes ago', '29 minutes ago',
       '5 hours ago', '8 hours ago', '6 hours ago', '9 hours ago',
       '11 hours ago', '12 hours ago', '7 hours ago', '10 hours ago'],
      dtype=object)

In [ ]:
# Создадим фукнцию для конвертации дат
def convert_to_date(s):
    # Текущая дата
    current_date = datetime.strptime('2023-05-23', '%Y-%m-%d')

    # Разобьем данные
    num, unit, *_ = s.split()
    num = int(num)

    # Вычтем время
    if unit.startswith('minute'):
        date = current_date - relativedelta(minutes=num)
    elif unit.startswith('hour'):
        date = current_date - relativedelta(hours=num)
    elif unit.startswith('day'):
        date = current_date - relativedelta(days=num)
    elif unit.startswith('week'):
        date = current_date - relativedelta(weeks=num)

    return date.date()

# Применим функцию
result_df['date'] = result_df['date'].apply(convert_to_date)

In [ ]:
# Конвертируем дату в datetime
result_df['date'] = pd.to_datetime(result_df['date'])
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   title            998 non-null    object        
 1   city             998 non-null    object        
 2   country          998 non-null    object        
 3   employment_type  930 non-null    object        
 4   company_name     969 non-null    object        
 5   employee_qty     964 non-null    object        
 6   company_field    964 non-null    object        
 7   skills           998 non-null    object        
 8   date             998 non-null    datetime64[ns]
 9   applicants       838 non-null    Int64         
dtypes: Int64(1), datetime64[ns](1), object(8)
memory usage: 79.1+ KB


In [ ]:
# Заменим пустые списки в 'skills' на np.nan
result_df['skills'] = result_df['skills'].apply(lambda x: np.nan if not x else x)

result_df.head()

,title,city,country,employment_type,company_name,employee_qty,company_field,skills,date,applicants
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting,"[data mining, excel, sap, sas, spss, sql, stat...",2023-05-16,47
1,Data Analyst - Logistics,Coventry,United Kingdom,On-site,NaN,NaN,NaN,NaN,2023-05-16,<NA>
2,Data Analyst - Logistics,Coventry,United Kingdom,On-site,NaN,NaN,NaN,[wfh],2023-05-16,<NA>
3,Data Analyst (Space & Planning),South Molton,United Kingdom,On-site,NaN,NaN,NaN,[excel],2023-05-16,<NA>
4,Data Analyst,Lugano,Switzerland,On-site,NaN,NaN,NaN,"[aws, data warehouse, etl, gcp, oracle, oracle...",2023-05-09,<NA>


## Подготовка данных к визуализации

1. Проверить данные на дубликаты и не релевантные заданию вакансии.

In [ ]:
# Проверим на дубликаты, но без столбца скиллы, тк это список
cols_except_skills = [col for col in result_df.columns if col != 'skills']

num_duplicates = result_df[cols_except_skills].duplicated().sum()

print(num_duplicates)

112


In [ ]:
# Удалите дубликаты
result_df = result_df.drop_duplicates(subset=cols_except_skills)

In [ ]:
# Сделаем список названий вакансий, которые нужно оставить
search_terms = ['data analyst', 'bi', 'business intelligence', 'data', 'Analista de datos', 'datos', 'Datový analytik', 'Business Analyst']
search_terms = [term.lower() for term in search_terms]

# Проверим если столбец title их содержит
contains_terms = result_df['title'].str.lower().apply(lambda title: any(term in title for term in search_terms))

# Оставим только те данные, которые имеют необходимые должности
result_df = result_df[contains_terms]
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 709 entries, 0 to 997
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   title            709 non-null    object        
 1   city             709 non-null    object        
 2   country          709 non-null    object        
 3   employment_type  654 non-null    object        
 4   company_name     682 non-null    object        
 5   employee_qty     677 non-null    object        
 6   company_field    677 non-null    object        
 7   skills           651 non-null    object        
 8   date             709 non-null    datetime64[ns]
 9   applicants       571 non-null    Int64         
dtypes: Int64(1), datetime64[ns](1), object(8)
memory usage: 61.6+ KB


Вывод: 998 было записей, осталось 709.

2. Подготовка датасета к визуализации

In [ ]:
# Используем метод explode для конвертации списка в столбце skills
tableau_df = result_df.reset_index(drop=True).explode('skills')
tableau_df.head()

,title,city,country,employment_type,company_name,employee_qty,company_field,skills,date,applicants
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting,data mining,2023-05-16,47
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting,excel,2023-05-16,47
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting,sap,2023-05-16,47
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting,sas,2023-05-16,47
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing & Recruiting,spss,2023-05-16,47


In [ ]:
# Загрузим CSV
tableau_df.to_csv('linkedin.csv', index=False)

## Ссылка на дашборд

Ссылка на дашборд: <https://public.tableau.com/shared/8BDS8T8BR?:display_count=n&:origin=viz_share_link>